In [78]:
#!/usr/bin/python3

import base64
import datetime
import requests

class Client():
    def __init__(self, username, password, endpoint='https://evo.elvaco.se/api/v1', ):
        self.endpoint = endpoint
        self.username = username
        self.password = password

        self.bearer_token = None

    def _authenticate(self):
        """
        API uses Basic Authentication
        """

        headers = {
            'authorization': 'Basic {0}'.format(
                base64.b64encode("{0}:{1}".format(
                    self.username,
                    self.password
                ).encode('utf-8')).decode('utf-8')
            )
        }

        url = "{0}/authenticate".format(self.endpoint)
        r = requests.get(url, headers=headers)

        authentication_response = r.json()

        if authentication_response.get('status', 200) != 200:
            # Exception ? authentication_response.get('message', None)
            return false

        self.bearer_token = authentication_response.get('token', None)

        return True

    def do_request(self, url, **kwargs):
        """
        Request wrapper
        """
        if self.bearer_token is None:
            self._authenticate()

        headers = {
            'authorization': 'Bearer {0}'.format(self.bearer_token)
        }

        r = requests.get(url, headers=headers, params=kwargs)
        data = r.json() # no longer returns json?

        if data.get('status', 200) != 200:
            # Exception ? authentication_response.get('message', None)
            return false

        return data

    def summary(self, medium='District heating'): # does not work anymore; returns some HTML
        url = '{0}/summary'.format(self.endpoint)
        after = datetime.datetime.now().isoformat()
        before = datetime.datetime.now().isoformat()
        reply = self.do_request(url, after=after, before=before, medium=medium)
        return reply

    def meters(self, size=50, page=0, medium='District heating'):
        url = '{0}/meters'.format(self.endpoint)
        after = '2019-11-01' # datetime.datetime.now().isoformat()
        before = '2019-12-01' # datetime.datetime.now().isoformat()
        medium = medium

        reply = self.do_request(url, after=after, before=before, medium=medium, size=size, page=page)
        # reply = self.do_request(url, medium=medium, size=size, page=page)
        return reply
        # return {
        #     'meters': reply.get('content', []),
        #     'totalPages': reply.get('totalPages', 0),
        #     'totalElements': reply.get('totalElements', 0),
        # }

    def meter(self, uuid):
        url = '{0}/meters/{1}'.format(self.endpoint, uuid)
        reply = self.do_request(url)
        return reply

In [79]:
client = Client("jens.brage@noda.se", "bV6CDHnC&tMm")

In [80]:
client.meters()

{'content': [{'id': '21c6582b-802f-4bf8-96bc-4c87682a991e',
   'facility': '121426',
   'address': '42545',
   'location': {'country': 'sverige',
    'city': 'växjö',
    'address': 'regementsgatan 4',
    'zip': 'unknown'},
   'medium': 'District heating',
   'manufacturer': 'ELV',
   'isReported': False,
   'collectionPercentage': 0.0,
   'organisationId': '0420446f-d32c-46f8-b2ec-8f9a6a1f4aec',
   'gatewaySerial': '12133603',
   'readIntervalMinutes': 60},
  {'id': 'd77ec9e7-8481-41d8-8489-cd2a583e432e',
   'facility': '121428',
   'address': '40238',
   'location': {'country': 'sverige',
    'city': 'växjö',
    'address': 'regementsgatan 8',
    'zip': '352 36',
    'position': {'latitude': 56.8776581827931,
     'longitude': 14.7922234766242,
     'confidence': 0.5}},
   'medium': 'District heating',
   'manufacturer': 'ELV',
   'isReported': False,
   'collectionPercentage': 0.0,
   'organisationId': '0420446f-d32c-46f8-b2ec-8f9a6a1f4aec',
   'gatewaySerial': '12101001',
   'rea

In [84]:
client.meter('d77ec9e7-8481-41d8-8489-cd2a583e432e')

{'id': 'd77ec9e7-8481-41d8-8489-cd2a583e432e',
 'facility': '121428',
 'address': '40238',
 'location': {'country': 'sverige',
  'city': 'växjö',
  'address': 'regementsgatan 8',
  'zip': '352 36',
  'position': {'latitude': 56.8776581827931,
   'longitude': 14.7922234766242,
   'confidence': 0.5}},
 'medium': 'District heating',
 'manufacturer': 'ELV',
 'statusChanged': '2018-05-08T12:58:18Z',
 'eventLog': [{'type': 'statusChange',
   'name': 'ok',
   'start': '2018-05-08T12:58:18Z'},
  {'type': 'newMeter', 'name': '40238', 'start': '2018-04-19T23:00:00Z'}],
 'created': '2018-04-21T01:28:45Z',
 'isReported': False,
 'alarms': [],
 'readIntervalMinutes': 1440,
 'organisationId': '0420446f-d32c-46f8-b2ec-8f9a6a1f4aec',
 'revision': 2,
 'mbusDeviceType': 4,
 'gateway': {'id': 'e415bb5a-86e9-46f2-b203-1bc21a9e5000',
  'productModel': 'CMi2110',
  'serial': '12101001',
  'status': {'id': 'ok', 'name': 'ok'},
  'statusChanged': '2018-05-08T12:58:18Z',
  'extraInfo': {}}}